In [ ]:
import faiss
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Load the index
index = faiss.read_index("ideas1.faiss")

print("Main index type:", type(index))

# For IndexIDMap, we need to access the underlying index
if isinstance(index, faiss.IndexIDMap):
    base_index = faiss.downcast_index(index.index)
    print("Base index type:", type(base_index))
    print("Base index attributes:", dir(base_index))
    
    # Try different methods to access vectors
    if hasattr(base_index, 'xb'):
        print("Index has xb attribute")
        vectors = np.array(base_index.xb).reshape(base_index.ntotal, base_index.d)
    elif hasattr(base_index, 'reconstruct_n'):
        print("Index has reconstruct_n method")
        vectors = np.empty((base_index.ntotal, base_index.d), dtype=np.float32)
        base_index.reconstruct_n(0, base_index.ntotal, vectors)
    elif hasattr(base_index, 'ivf'):
        print("Index has ivf attribute")
        invlists = base_index.invlists
        vectors = []
        for i in range(base_index.nlist):
            list_size = invlists.list_size(i)
            if list_size > 0:
                list_vectors = faiss.rev_swig_ptr(invlists.get_codes(i), list_size * base_index.code_size)
                vectors.extend(list_vectors)
        vectors = np.array(vectors, dtype=np.float32)
    else:
        print("Could not find a way to access vectors. Available attributes:", dir(base_index))
        raise ValueError("Cannot access vectors from this index type")
else:
    print("Not an IndexIDMap")
    raise ValueError("Unexpected index type")

print("Vector shape:", vectors.shape)

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Plot
plt.figure(figsize=(10, 8))
plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], s=30, alpha=0.7)
plt.title("FAISS Vector Visualization with t-SNE", fontsize=14)
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.show()